In [1]:
from datasets import load_dataset

ds = load_dataset("TIGER-Lab/MMLU-Pro")

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

# 특정 스플릿 선택 (예: train 스플릿)
testds = ds["test"]

# pandas DataFrame으로 변환
mmlupro = testds.to_pandas()

mmlupro = mmlupro[["question", "options", 'answer', 'category']]
mmlupro

,question,options,answer,category
0,"Typical advertising regulatory bodies suggest,...","[Safe practices, Fear, Jealousy, Trivial, Unsa...",I,business
1,Managers are entrusted to run the company in t...,"[Shareholders, Diligence, Self-interest, Share...",F,business
2,There are two main issues associated with ____...,"[Down, Autonomy, Remuneration, Benefit, Down, ...",J,business
3,_______ locate morality beyond the sphere of r...,"[Ethical egoism, Ethics of duty, Postmodern et...",C,business
4,Some of key differences between Islamic finan...,"[Interest, Certain, Assured, Both tangible and...",G,business
...,...,...,...,...
12027,A hot mild steel rod is placed in a carbonaceo...,"[2.2 hr, 0.75 hr, 1.5 hr, 6.0 hr, 4.5 hr, 0.30...",J,engineering
12028,The cost of making the correct decisions for t...,"[(a) Decision made is d_1, (b) Decision d_1, (...",H,engineering
12029,Consider the evaporation of liquid ammonia int...,"[110 lb moles/hr-ft^2-atm, 200 lb moles/hr-ft^...",F,engineering
12030,"Air (100°F, 1atm) is flowing at a velocity of ...",[Local convective heat transfer coefficient: 1...,I,engineering


# ------------------------------------------------------
### 토픽별 

In [ ]:
# 원하는 토픽 지정 !!!!!!!!!!!!!!!
topic = 'business'
mmlupro_select = mmlupro[mmlupro['category']==topic]
mmlupro_select.reset_index(drop=True, inplace=True) # 인덱스 초기화
mmlupro_select['quesNum'] = None
for idx in range(len(mmlupro_select)) : mmlupro_select.loc[idx, 'quesNum'] = f'QUESTION{idx}) '
mmlupro_select

In [ ]:
# 알파벳을 붙이는 작업
for idx in range(len(mmlupro_select)) : 


    mmlupro_select.at[idx, 'options'] = [(f"({chr(65 + i)}) {item}") for i, item in enumerate(mmlupro_select['options'][idx])]
    mmlupro_select.at[idx, 'options'] = '\n'.join(mmlupro_select['options'][idx])

# 출력 확인
mmlupro_select


In [ ]:
mmlupro_select["prompts"] = mmlupro_select['quesNum'] + mmlupro_select["question"] + "\n" + mmlupro_select["options"]
# 큰따옴표 제거 (혹시나)
mmlupro_select = mmlupro_select.replace("\"", "\'", regex=True)
#csv로 저장
mmlupro_select = mmlupro_select.rename(columns={'answer': 'answers'})
mmlupro_select[['prompts', 'answers']].to_csv(data_path+f'mmlupro_test_{topic}.csv', index = False, sep=',')

# -------------------------------------------------------
### 카테고리 넣은 버전

In [3]:
mmlupro_mix = mmlupro[mmlupro['category'].isin(['philosophy', 'business', 'history', 'law', 'psychology'])]
mmlupro_mix

,question,options,answer,category
0,"Typical advertising regulatory bodies suggest,...","[Safe practices, Fear, Jealousy, Trivial, Unsa...",I,business
1,Managers are entrusted to run the company in t...,"[Shareholders, Diligence, Self-interest, Share...",F,business
2,There are two main issues associated with ____...,"[Down, Autonomy, Remuneration, Benefit, Down, ...",J,business
3,_______ locate morality beyond the sphere of r...,"[Ethical egoism, Ethics of duty, Postmodern et...",C,business
4,Some of key differences between Islamic finan...,"[Interest, Certain, Assured, Both tangible and...",G,business
...,...,...,...,...
11058,Mill claims that in deciding how to act:,"[we should consult tradition., we should alway...",C,philosophy
11059,Of the two versions of the principle that Sing...,[he considers the stronger version to be corre...,A,philosophy
11060,"According to Kant, laws of nature are laws acc...",[nothing will happen; everything ought to happ...,C,philosophy
11061,Hare refers to people who endorse ideals witho...,"[fanatics., irrationalists., purists., non-pra...",A,philosophy


In [22]:
# 행 순서 섞기
mmlupro_mix = mmlupro_mix.sample(frac=1).reset_index(drop=True)
mmlupro_mix

,prompts,answers,category
0,QUESTION2969) A drama guild is renovating an o...,C,business
1,QUESTION1864) The incidence of schizophrenia i...,B,psychology
2,QUESTION2992) Use indirect truth tables to det...,A,philosophy
3,QUESTION2644) A law student and two friends we...,C,law
4,QUESTION594) What distinctions are made when d...,A,psychology
...,...,...,...
3563,QUESTION1968) Pinson's Wholesale Hardware Comp...,G,business
3564,QUESTION1792) Which of the following will not ...,D,law
3565,"QUESTION3300) During a three-month period, a c...",G,law
3566,QUESTION1565) Objects that absorb light appear...,C,psychology


In [5]:
mmlupro_mix['quesNum'] = None
for idx in range(len(mmlupro_mix)) : mmlupro_mix.loc[idx, 'quesNum'] = f'QUESTION{idx}) '
mmlupro_mix

,question,options,answer,category,quesNum
0,"By analyzing the levels of 13C in bones, resea...","[a diet high in calcium or low in calcium., a ...",C,history,QUESTION0)
1,A man was on a date when he realized that he d...,"[not guilty, because he did not intend to burn...",E,law,QUESTION1)
2,A truck and a car collided at an intersection....,"[The truck driver recovers $4,000, and the car...",H,law,QUESTION2)
3,"A father lived with his son, who was addicted ...","[Give the self-defense instruction, because th...",G,law,QUESTION3)
4,Jane Morrow gave a 60-day sight-draft dated Ma...,"[$251.94, $256.50, $242.75, $252.94, $254.94, ...",J,business,QUESTION4)
...,...,...,...,...,...
3563,"For many years, persons engaged in real estate...","[No, because application of the law to install...",D,law,QUESTION3563)
3564,Which of the following concepts can be defined...,"[all of the above, bad, good, green, unicorn, ...",J,philosophy,QUESTION3564)
3565,Weiser Inc. is offering a discount of $21.90 o...,"[List price: $175.20, Net price: $153.30, List...",A,business,QUESTION3565)
3566,A city ordinance forbids washing vehicles park...,[The man had already planned to dry the water ...,J,law,QUESTION3566)


In [6]:
# 알파벳을 붙이는 작업
for idx in range(len(mmlupro_mix)) : 


    mmlupro_mix.at[idx, 'options'] = [(f"({chr(65 + i)}) {item}") for i, item in enumerate(mmlupro_mix['options'][idx])]
    mmlupro_mix.at[idx, 'options'] = '\n'.join(mmlupro_mix['options'][idx])

# 출력 확인
mmlupro_mix

,question,options,answer,category,quesNum
0,"By analyzing the levels of 13C in bones, resea...",(A) a diet high in calcium or low in calcium.\...,C,history,QUESTION0)
1,A man was on a date when he realized that he d...,"(A) not guilty, because he did not intend to b...",E,law,QUESTION1)
2,A truck and a car collided at an intersection....,"(A) The truck driver recovers $4,000, and the ...",H,law,QUESTION2)
3,"A father lived with his son, who was addicted ...","(A) Give the self-defense instruction, because...",G,law,QUESTION3)
4,Jane Morrow gave a 60-day sight-draft dated Ma...,(A) $251.94\n(B) $256.50\n(C) $242.75\n(D) $25...,J,business,QUESTION4)
...,...,...,...,...,...
3563,"For many years, persons engaged in real estate...","(A) No, because application of the law to inst...",D,law,QUESTION3563)
3564,Which of the following concepts can be defined...,(A) all of the above\n(B) bad\n(C) good\n(D) g...,J,philosophy,QUESTION3564)
3565,Weiser Inc. is offering a discount of $21.90 o...,"(A) List price: $175.20, Net price: $153.30\n(...",A,business,QUESTION3565)
3566,A city ordinance forbids washing vehicles park...,(A) The man had already planned to dry the wat...,J,law,QUESTION3566)


In [ ]:
mmlupro_mix["prompts"] = mmlupro_mix['quesNum'] + mmlupro_mix["question"] + "\n" + mmlupro_mix["options"]
# 큰따옴표 제거 (혹시나)
mmlupro_mix = mmlupro_mix.replace("\"", "\'", regex=True)
#csv로 저장
mmlupro_mix = mmlupro_mix.rename(columns={'answer': 'answers'})
# mmlupro_mix[['prompts', 'answers','category']].to_csv(data_path+f'mmlupro_test_mix.csv', index = False, sep=',')

# -----------------------------------------_
### 학칙이랑 mmlu 죄다 섞은 FULL 버전

In [12]:
mmlupro_mix = mmlupro_mix[['prompts', 'answers','category']]
mmlupro_mix

,prompts,answers,category
0,QUESTION0) By analyzing the levels of 13C in b...,C,history
1,QUESTION1) A man was on a date when he realize...,E,law
2,QUESTION2) A truck and a car collided at an in...,H,law
3,"QUESTION3) A father lived with his son, who wa...",G,law
4,QUESTION4) Jane Morrow gave a 60-day sight-dra...,J,business
...,...,...,...
3563,"QUESTION3563) For many years, persons engaged ...",D,law
3564,QUESTION3564) Which of the following concepts ...,J,philosophy
3565,QUESTION3565) Weiser Inc. is offering a discou...,A,business
3566,QUESTION3566) A city ordinance forbids washing...,J,law


In [19]:
ewha = pd.read_csv(data_path+'test_own_ewha.csv')
ewha['category'] = 'ewha'

In [25]:
full_test_samples = pd.concat([mmlupro_mix, ewha], ignore_index=True)
full_test_samples = full_test_samples.sample(frac=1).reset_index(drop=True)

In [26]:
full_test_samples

,prompts,answers,category
0,QUESTION787) Discuss the influence of the five...,E,psychology
1,QUESTION1172) A husband came home early from w...,A,law
2,QUESTION310) The Younger Dryas was:\n(A) a per...,C,history
3,QUESTION354) Homo habilis emerged around _____...,B,history
4,QUESTION2368) Mrs. Reynolds purchased $450 wor...,J,business
...,...,...,...
3605,QUESTION794) A buyer contracted in writing to ...,A,law
3606,QUESTION564) A sister and brother own a ranch ...,E,law
3607,QUESTION3445) For deception in an experiment t...,D,psychology
3608,QUESTION1995) A woman was married to a very we...,A,law


In [27]:
full_test_samples.to_csv(data_path+f'full_test_samples.csv', index = False, sep=',')